DNN과 다르게 RNN에서는 okt형태소 토큰화 후 워드임베딩을 통해 문장의 문맥을 파악하는 방식으로 학습 시킬것이다. 
일반적으로 불용어를 제거=> 노이즈를 줄여서 성능향상을 도모
그러나 감성분류 5가지에선 불용어 제거하지않고 형태소를 분리하고 어절을 그대로 사용한다.

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

#한국어 감성사전 불러온다.
with open('./SentiWord_Dict.txt', 'r', encoding='utf-8') as f:
    SentiWord_Dict = f.read()

df_train = pd.DataFrame({
    'feature': [line.split('\t')[0] for line in SentiWord_Dict.split('\n') if len(line.split('\t')) >= 2],
    'target': [int(line.split('\t')[1]) for line in SentiWord_Dict.split('\n') if len(line.split('\t')) >= 2]
})

In [2]:
#문장 추출
texts= [ ]
for i in df_train['feature']:
    texts.append(i)
len(texts)
# texts

14856

In [3]:
#문자열이 아닌게 있는지 확인
for i in texts:
    if type(i)==str:
        print(i)

(-;
(;_;)
(^^)
(^-^)
(^^*
(^_^)
(^_^;
(^o^)
(-_-)
(T_T)
(ㅡㅡ)
)-:
*^^*
/^o^\
:(
:'-(
:-(
:)
:-)
:/
:|
:D
:-D
:p
:-P
;
;)
;-)
;ㅅ;
^///^
^^
^^//
^-^;
^-^;;
^-^;;;;;
-_-^
<3
=)
8-)
B-)
XD
ㄱㅅ
ㄱㅇㄷ
가격이 싸다
가까스로
가까이 사귀어
가까이하다
가꾸러뜨리다
가꾸러트리다
가난
가난뱅이
가난살이
가난살이하다
가난설음
가난에
가난에 쪼들려서
가난하게
가난하고
가난하고 어렵다
가난하고 천한
가난하다
가난하여
가난한
가난한 데다가
가난한 사람을
가난히
가년스럽다
가념하다
가녕스럽다
가누기 어려워하다
가누지 못하고
가누지 못하다
가누지 못할
가누지 못할 정도로
가누지 못함
가눌 수 없게
가눌 수 없을
가눌 수 없이
가능성이 늘어나다
가능성이 있다고
가능하다
가닐가닐하다
가다듬어
가다듬어 수습하는
가다듬어 수습하다
가다듬어 정하다
가당찮다
가당찮이
가당히
가두거나
가두거나 해치거나
가드올려
가뜬하게
가라앉다
가라앉지 않은
가라앉혀 바로잡다
가래
가래 따위가
가래가
가래가 섞인
가량맞고
가량맞고 야살스러운
가량맞고 야살스러움
가려서 좋아하다
가려운
가려운 느낌이
가려운 증상을
가련하게
가련하게 여기다
가련하게 여김
가련한
가련한 듯하다
가렵고
가로막아서
가로채서
가로채어
가르치며
가르침
가르침을 받아
가리지 못하다
가리지 않고
가마득하다
가만있지 못하고
가망 없는
가망이 없게
가벼운 마음으로
가벼운 말이나
가벼운 상처
가벼운 슬픔
가벼운 행동으로
가볍게
가볍게 까부는
가볍게 까불다
가볍게 쑤시듯
가볍게 웃다
가볍게 채신없이
가볍게 행동하거나
가볍게 행동하는
가볍게 행동하다
가볍고
가볍고 보드랍게
가볍고 상쾌하다
가볍고 상쾌한
가볍고 시원하게
가볍고 편안하게
가볍고 환하게
가분가분
가분히
가뿐가뿐
가뿐가뿐하다
가뿐가뿐히
가뿐하게
가뿐하다
가뿐한
가뿐한 느낌
가뿐한 느낌이
가뿐히
가쁘게
가쁘게 쉬다
가쁘고
가쁘고 거칠게
가쁘고 급하게
가

In [4]:
df_train

,feature,target
0,(-;,1
1,(;_;),-1
2,(^^),1
3,(^-^),1
4,(^^*,1
...,...,...
14851,내팽개치다,-2
14852,횡령,-2
14853,불안증,-2
14854,어쩔티비,-1


In [5]:
#라벨인코딩 하지않고 해볼것이다.
# from sklearn.preprocessing import LabelEncoder 

# #LabelEncoder 객체를 생성
# label_encoder = LabelEncoder()

# #타겟 변수를 인코딩
# encoded_target = label_encoder.fit_transform(df_train['target'])

# #인코딩된 타겟 변수를 원래 데이터프레임에 추가
# df_train['encoded_target'] = encoded_target
# decoded_target = label_encoder.inverse_transform(encoded_target)

In [6]:
# 라벨별로 분류 => 이거 시각화해보면 될듯
very_bad = df_train[df_train['target'] == -2]
bad = df_train[df_train['target'] == -1]
soso = df_train[df_train['target'] == 0]
good = df_train[df_train['target'] == 1]
very_good = df_train[df_train['target'] == 2]


In [7]:
#데이터 불균형 확인
print(df_train['target'][df_train['target']==-2].count())
print(df_train['target'][df_train['target']==-1].count())
print(df_train['target'][df_train['target']==0].count())
print(df_train['target'][df_train['target']==1].count())
print(df_train['target'][df_train['target']==2].count())


4799
5031
154
2269
2603


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#tokenized_text의 빈도수 상위 만개 어절을 토크나이저에 fit시킴
# filters='' 를 추가하여 기호필터링을 제거해준다.
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(df_train['feature'])

#단어 빈도수 시각화
#불용어제거후 시각화해볼수도있음

#시퀀스데이터로 변환 ==벡터화
sequences = tokenizer.texts_to_sequences(df_train['feature'])

sequences

[[4457],
 [4458],
 [4459],
 [4460],
 [4461],
 [4462],
 [4463],
 [4464],
 [4465],
 [4466],
 [4467],
 [4468],
 [4469],
 [4470],
 [4471],
 [4472],
 [4473],
 [4474],
 [4475],
 [4476],
 [4477],
 [4478],
 [4479],
 [4480],
 [4481],
 [4482],
 [4483],
 [4484],
 [4485],
 [4486],
 [4487],
 [4488],
 [4489],
 [4490],
 [4491],
 [4492],
 [4493],
 [4494],
 [4495],
 [4496],
 [4497],
 [4498],
 [4499],
 [4500, 4501],
 [4502],
 [4503, 2241],
 [4504],
 [4505],
 [4506],
 [4507],
 [4508],
 [4509],
 [4510],
 [4511],
 [2242],
 [2242, 2243],
 [4512],
 [799],
 [799, 457],
 [799, 458],
 [4513],
 [1244],
 [800],
 [800, 4514],
 [800, 22],
 [4515],
 [4516],
 [4517],
 [4518],
 [4519, 1245],
 [459, 5],
 [459, 16],
 [459, 286],
 [459, 286, 42],
 [459, 171],
 [1246, 68, 287],
 [1246, 68, 344],
 [1246, 68, 6],
 [1247, 1248],
 [1247, 4520],
 [4521],
 [4522],
 [345],
 [345, 1249],
 [345, 2244],
 [345, 2245],
 [4523],
 [4524],
 [4525],
 [1250],
 [1250, 1251],
 [4526],
 [4527],
 [4528],
 [4529, 63],
 [4530, 801],
 [2246],
 [

In [9]:
#최대길이 확인
max_len = max([len(i) for i in sequences])
max_len

8

In [10]:
#길이를 맞춰주는 패딩작업,
x_train = pad_sequences(sequences, maxlen=max_len, padding='post')

In [11]:
#y값 추출, 데이터 차원 맞추기, 넘파이 배열로 변환
y_train=(df_train['target'].to_frame()).to_numpy()+2
y_train

array([[3],
       [1],
       [3],
       ...,
       [0],
       [1],
       [3]], dtype=int64)

In [12]:
#시퀀스 길이확인
print(x_train[0])
x_train.shape

[4457    0    0    0    0    0    0    0]


(14856, 8)

In [13]:
#단어수 확인
word_count = len(tokenizer.word_index) + 1
word_count

9744

In [18]:
# GRU 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding,BatchNormalization,Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop

model = Sequential([
    Embedding( word_count, 300, input_length=max_len ),
    Bidirectional(GRU(32)),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

# 모델 컴파일
optimizer= Adam(learning_rate =0.003)
# optimizer= RMSprop(learning_rate=0.001)
model.compile(optimizer=optimizer , loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#콜백정의
ES =EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)

#모델학습
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.25, callbacks=[ES])


Epoch 1/20
88/88 [==============================] - 18s 102ms/step - loss: 1.3173 - accuracy: 0.4356 - val_loss: 1.4332 - val_accuracy: 0.2771
Epoch 2/20
88/88 [==============================] - 7s 78ms/step - loss: 0.6977 - accuracy: 0.7496 - val_loss: 1.3264 - val_accuracy: 0.3850
Epoch 3/20
88/88 [==============================] - 6s 64ms/step - loss: 0.4428 - accuracy: 0.8542 - val_loss: 1.6544 - val_accuracy: 0.3538
Epoch 4/20
88/88 [==============================] - 6s 71ms/step - loss: 0.3391 - accuracy: 0.8888 - val_loss: 2.0420 - val_accuracy: 0.3958
Epoch 5/20
88/88 [==============================] - 6s 68ms/step - loss: 0.2768 - accuracy: 0.9112 - val_loss: 2.2064 - val_accuracy: 0.4152
Epoch 6/20
88/88 [==============================] - 6s 66ms/step - loss: 0.2193 - accuracy: 0.9314 - val_loss: 2.0636 - val_accuracy: 0.4532


In [19]:
#테스트문장 입력
test = ['회식끝나고 집가는중','개꿀잼이네','진짜 너무 별로다.','ㅋㅋㅋㅋㅋㅋㅋㅋ']


#mecab
# from konlpy.tag import Mecab
# tokenized_test_text = [ mecab.morphs(i) for i in clean_test_texts]

# #벡터화
test_sequences = tokenizer.texts_to_sequences(test) 
x_test = pad_sequences(test_sequences,padding='post',maxlen=max_len) 

# #예측
prediction = model.predict(x_test)  

for i in prediction:
    print(i)

#학습데이터내에 없는 단어들이 들어와서 
#시퀀스 데이터가 생성되지 않았기 때문에 감성예측이 되지 않았다.

1/1 [==============================] - 1s 1s/step
[0.3469524  0.11255463 0.04070368 0.3505402  0.14924912]
[0.3469524  0.11255463 0.04070368 0.3505402  0.14924912]
[0.48670805 0.17821309 0.0377988  0.18668741 0.11059266]
[0.3469524  0.11255463 0.04070368 0.3505402  0.14924912]


In [ ]:
# model.save('기본 DNN 적용모델.h5')